# Migating Generative AI with Avanced Prompt Engineering
Copyright 2023 Denis Rothman, MIT License

The goal of this notebook is to introduce some Generative AI mitigating tools for educational purposes. Each tools is presented separately to provide ideas that can be expanded as deemed fit for a project or simply consulted.




#Prompt Engineering

Advanced prompt engineering is not limited to entering a good prompt or question. This notebook will show you how to implement a transformer efficiently with advanced prompt engineering:<br>

Moderation, quality control<br>

1.& 5. Input/output Moderation   
2.Building the Knowledge base(KB)<br>
3.Parsing the user's prompt and accessing the KB<br>
4.Generating ChatGPT content with a dialog function and Token management<br>






### Installing OpenAI

In [1]:
#Importing openai
try:
  import openai
except:
  !pip install openai
  import openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.3 MB/s eta 0:00:00


###Your API Key

In [2]:
#2.API Key
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
from google.colab import drive
drive.mount('/content/drive')
f = open("drive/MyDrive/files/api_key.txt", "r")
API_KEY=f.readline()
f.close()

#The OpenAI Key
import os
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

Mounted at /content/drive


#1.& 5.Input and output moderation with transformers and rule-bases

Pre-processing prompts and post-processing completions

OpenAI provides[ tools](https://https://openai.com/blog/new-and-improved-content-moderation-tooling) to control the input and output flow of their models.

In [3]:
def moderator(text):
  mod_response = openai.Moderation.create(
  input=text,
  model='text-moderation-latest')
  return mod_response

In [4]:
text="This is a good sentence but a distasteful topic."

In [5]:
def rule_base(text):
    words = ['bad', 'distasteful', 'evil', 'unproductive', 'null']

    for word in words:
        if word in text:
            print("Input flagged: True")
            return
    print("Input flagged: False")



In [6]:
# Test the function
rule_base("This is a distasteful example")
rule_base("This is a good example")


Input flagged: True
Input flagged: False


In [7]:
response=moderator(text)

Displaying the details of the response(JSON object)

In [8]:
response["results"][0]["flagged"]

False

In [9]:
response["results"][0]["categories"]

<OpenAIObject at 0x7fea2de0a7a0> JSON: {
  "sexual": false,
  "hate": false,
  "harassment": false,
  "self-harm": false,
  "sexual/minors": false,
  "hate/threatening": false,
  "violence/graphic": false,
  "self-harm/intent": false,
  "self-harm/instructions": false,
  "harassment/threatening": false,
  "violence": false
}

In [10]:
response["results"][0]["category_scores"]

<OpenAIObject at 0x7fea2de292b0> JSON: {
  "sexual": 8.360543e-05,
  "hate": 3.7862097e-07,
  "harassment": 0.00039490603,
  "self-harm": 2.1369965e-07,
  "sexual/minors": 1.2816221e-06,
  "hate/threatening": 2.2716094e-10,
  "violence/graphic": 2.1425122e-07,
  "self-harm/intent": 7.2579206e-07,
  "self-harm/instructions": 1.1938396e-07,
  "harassment/threatening": 2.7388277e-08,
  "violence": 5.2087296e-05
}

Flagged as sensitive "True" or "False"

# 2.Building a knowledge base for ChatGPT, GPT-4


In [11]:
assert1={'role': 'assistant', 'content': 'Opening hours of Snap-LM Consulting :Monday through Friday 9am to 5pm. Services :expert systems, rule-based systems, machine learning, deep learning, transformer models.'}
assert2={'role': 'assistant', 'content': 'Services :expert systems, rule-based systems, machine learning, deep learning, transformer models.'}
assert3={'role': 'assistant', 'content': 'Services :Fine-tuning OpenAI GPT-3 models, designing datasets, designing knowledge bases.'}
assertn={'role': 'assistant', 'content': 'Services:advanced prompt engineering using a knowledge base and SEO keyword methods.'}

#Using the knowledge base as a dataset:
kbt = []
kbt.append(assert1)
kbt.append(assert2)
kbt.append(assert3)
kbt.append(assertn)

In [12]:
#displaying the KB as a DataFrame(DF) Clic on the magic Google Colaboratory Wand to obtain a cool display
import pandas as pd
df=pd.DataFrame(kbt)
df

,role,content
0,assistant,Opening hours of Snap-LM Consulting :Monday th...
1,assistant,"Services :expert systems, rule-based systems, ..."
2,assistant,"Services :Fine-tuning OpenAI GPT-3 models, des..."
3,assistant,Services:advanced prompt engineering using a k...


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


Let's add some metadata keywords for each record of the knowledge base

In [13]:
assertkw1="open"
assertkw2="expert"
assertkw3="services"
assertkwn="prompt"

In [14]:
#create a kb keywords as list
kbkw=[assertkw1,assertkw2,assertkw3,assertkwn]
#displaying the KB as a DataFrame(DF) Clic on the magic Google Colaboratory Wand to obtain a cool display
dfk=pd.DataFrame(kbkw)
dfk

,0
0,open
1,expert
2,services
3,prompt


#3.Parsing the user requests and accessing the KB







In [15]:
user_requests=[]
user_requests.append({'role': 'user', 'content': 'At what time does Snap-LM Consulting open on Monday?'})
user_requests.append({'role': 'user', 'content': 'At what time does Snap-LM Consulting open on Saturday?'})
user_requests.append({'role': 'user', 'content': 'Can you create an AI-driven expert system?'})
user_requests.append({'role': 'user', 'content': 'What services does Snap-LM Consulting offer?'})

In [16]:
n=len(user_requests)

In [17]:
# This is an example. You can customize this as you wish for your project
def parse_user(uprompt,kbkw,kbt):
  i=0
  j=0
  for kw in kbkw:
    #print(i,kw)
    rq=str(uprompt)
    k=str(kw)
    fi=rq.find(k)
    if fi>-1:
      print(kw,rq,kbt[i])
      j=i
    i+=1
  return kbt[j]


#4.Generating ChatGPT content with a dialog function

Generating content with a function that fits the user's request using OpenAI ChatGPT through the gpt-3.5 turbo model.

This function will receive an automated prompt.


In [18]:
#convmodel="gpt-3.5-turbo"
convmodel="gpt-4"
def dialog(iprompt):
    response = openai.ChatCompletion.create(
        model=convmodel,
        messages=iprompt
    )
    return response

## Implementing ChatGPT with a simplified search engine approach

The following cell contains 6 steps: <br>

Step 1: iterating through the user's requests   <br>
Step 2: the application goes through a the user's request and searches for keywords with a search-engine-like technique<br>
Step 3: the application creates a prompt, with a system message, the knowledge base record found and the initial user's request <br>
Step 4: The prompt is sent to the ChatGPT dialog function<br>
Step 5: storing the response in a list<br>
Step 6; displaying the KB as a DataFrame(DF);click on the magic Google Colaboratory Wand to obtain a cool display


5. Post-Processsing Moderation

See section 1.Moderation with transformers dans rule-bases

In [19]:
responses=[]      #creating a list to store the dialog

#going through the user's requests in a batch for the ChatGPT simulation
for i in range(n):
  # Step 1: iterating through the user's requests
  user_request_num=i

  #Step 2: the application goes through a the user's request and searches for keywords with a search-engine-like technique
  #        to find a record in the knowledge base
  kb_record=parse_user(user_requests[user_request_num],kbkw,kbt)

  #Step 3: the application creates a prompt, with a system message, the knowledge base record found and the initial user's request
  iprompt = []
  iprompt.append({"role": "system", "content": "You are an assistant for Snap-LM Consulting."})
  iprompt.append(kb_record)
  iprompt.append(user_requests[user_request_num])

  #print(iprompt)

  #Step 4: The prompt is sent to the ChatGPT dialog function
  response = dialog(iprompt)

  #Step 5: storing the response in a list
  ex=response["choices"][0]["message"]["content"]
  rt="Total Tokens:" + str(response["usage"]["total_tokens"])
  responses.append([user_requests[user_request_num],ex,rt])

#Step 6; displaying the KB as a DataFrame(DF);click on the magic Google Colaboratory Wand to obtain a cool display
pd.DataFrame(responses, columns=['request', 'response', 'tokens'])

open {'role': 'user', 'content': 'At what time does Snap-LM Consulting open on Monday?'} {'role': 'assistant', 'content': 'Opening hours of Snap-LM Consulting :Monday through Friday 9am to 5pm. Services :expert systems, rule-based systems, machine learning, deep learning, transformer models.'}
open {'role': 'user', 'content': 'At what time does Snap-LM Consulting open on Saturday?'} {'role': 'assistant', 'content': 'Opening hours of Snap-LM Consulting :Monday through Friday 9am to 5pm. Services :expert systems, rule-based systems, machine learning, deep learning, transformer models.'}
expert {'role': 'user', 'content': 'Can you create an AI-driven expert system?'} {'role': 'assistant', 'content': 'Services :expert systems, rule-based systems, machine learning, deep learning, transformer models.'}
services {'role': 'user', 'content': 'What services does Snap-LM Consulting offer?'} {'role': 'assistant', 'content': 'Services :Fine-tuning OpenAI GPT-3 models, designing datasets, designing 

,request,response,tokens
0,"{'role': 'user', 'content': 'At what time does...",Snap-LM Consulting opens at 9am on Mondays.,Total Tokens:86
1,"{'role': 'user', 'content': 'At what time does...","I'm sorry for the confusion, but Snap-LM Consu...",Total Tokens:109
2,"{'role': 'user', 'content': 'Can you create an...","Yes, as an AI, I can aid in the process of cre...",Total Tokens:228
3,"{'role': 'user', 'content': 'What services doe...",Snap-LM Consulting offers a range of services ...,Total Tokens:280
